In [ ]:
## mashup script for batch changes
import os, osgeo
# del os.environ['PROJ_LIB']
import math
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
from geopandas import GeoDataFrame
from pandas import DataFrame
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
from pyproj import proj
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum
import rasterio
from rasterstats import zonal_stats
import xarray as xr
import numpy as np
from osgeo import gdal, gdalconst, osr
import sys, time, ogr
import libpysal
gdal.UseExceptions()  # not required, but a good idea
gdal.AllRegister()
# Allow division by zero
numpy.seterr(divide='ignore', invalid='ignore')

In [ ]:
# Raster dataset
infile = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048.tif"
outfile = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048-bin-1.tif"
# datascr = rasterio.open(infile)
# out_profile = datascr.profile.copy()
# dst = rasterio.open(outfile, 'w', **out_profile)

In [ ]:
# this creates a binary raster beaased ont he threshold
def process_image(src_img, dst_img, band_id=1, threshold=0.2):
    with rasterio.open(src_img) as src:
        kwargs = src.meta
        tiles = src.block_windows(band_id)
        profile = src.profile
        # print(profile)
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw')
        with rasterio.open(dst_img, 'w', **profile) as dst:
            for idx, window in tiles:
                print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
                dst_data = src.read(band_id, window=window)
                ndval = src.nodatavals[0]
                print(ndval)
                dst_data[dst_data==ndval] = np.nan
                dst_data[dst_data<threshold]=0
                dst_data[dst_data>=threshold]=1
                # dst_data = src_data ** 2 # Do the Processing Here
                dst.write_band(band_id, dst_data, window=window)
    return

In [ ]:
process_image(infile,outfile)

In [ ]:
bin_img = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048-bin.tif"
with rasterio.open(bin_img) as src:
        kwargs = src.meta
        tiles = src.block_windows(1)
        profile = src.profile
        # print(profile)
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw')

        dest = gpd.GeoDataFrame(columns=['class', 'feature'], geometry='feature',crs = src.crs)

        for idx, window in tiles:
            print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
            data = src.read(1, window=window)
            ndval = src.nodatavals[0]
            data[data==ndval] = np.nan
            data[data==0] = np.nan
            # window_transform(window)

            # Use a generator instead of a list
            # shape_gen = ((shape(s), v) for s, v in shapes(data, transform=src.transform))
            shape_gen = ((shape(s), v) for s, v in shapes(data, transform=src.window_transform(window)))

            # either build a pd.DataFrame
            # df = DataFrame(shape_gen, columns=['geometry', 'class'])
            # gdf = GeoDataFrame(df["class"], geometry=df.geometry, crs=src.crs)

            # or build a dict from unpacked shapes
            gdf = GeoDataFrame(dict(zip(["geometry", "class"], zip(*shape_gen))), crs=src.crs)
            gdf = gdf[gdf['class']==1]
            dest = pd.concat([dest, gdf])

In [ ]:
final= gpd.GeoDataFrame(dest['geometry'])
ndvi_vec = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048-vec.gpkg"
final.to_file(ndvi_vec, driver='GPKG', layer='2022ndvi')

In [ ]:
ruh_urb = r"C:\Users\DroneAdmin\Downloads\Urban-Environs-limit.shp"
ruh = gpd.read_file(ruh_urb)

In [ ]:
result = gpd.overlay(ruh, final, how='intersection')

In [ ]:
print(result.area.sum()/ruh.area*100)
#result['area'] =result.apply(lambda row: row.geometry.area,axis=1)

In [ ]:
import libpysal

# create spatial weights matrix
W = libpysal.weights.Queen.from_dataframe(final)

# get component labels
components = W.component_labels

combined_polygons = gdf.dissolve(by=components)

In [ ]:
# Preprocessing based on worldview 3 meta data. This has to be adjusted per image strip
# xmldoc = minidom.parse("/Users/philipp/BPLA Dropbox/08 GIS-data/GIS-data-Riyadh/2021-WorldView3-8BAnds/21MAY13073945-S2AS-RCRC-8Bands.XML")
# nodes = xmldoc.getElementsByTagName("IMD")
#
#
# document = parse("/Users/philipp/BPLA Dropbox/08 GIS-data/GIS-data-Riyadh/2021-WorldView3-8BAnds/21MAY13073945-S2AS-RCRC-8Bands.XML")
# IMD = document.find('IMD')
#
# bandC_F = IMD.find('BAND_C').findtext('ABSCALFACTOR')
# bandB_F = IMD.find('BAND_B').findtext('ABSCALFACTOR')
# bandG_F = IMD.find('BAND_G').findtext('ABSCALFACTOR')
# bandY_F = IMD.find('BAND_Y').findtext('ABSCALFACTOR')
# bandR_F = IMD.find('BAND_R').findtext('ABSCALFACTOR')
# bandRE_F = IMD.find('BAND_RE').findtext('ABSCALFACTOR')
# bandN_F = IMD.find('BAND_N').findtext('ABSCALFACTOR')
# bandN2_F = IMD.find('BAND_N2').findtext('ABSCALFACTOR')
# bandC_E = IMD.find('BAND_C').findtext('EFFECTIVEBANDWIDTH')
# bandB_E = IMD.find('BAND_B').findtext('EFFECTIVEBANDWIDTH')
# bandG_E = IMD.find('BAND_G').findtext('EFFECTIVEBANDWIDTH')
# bandY_E = IMD.find('BAND_Y').findtext('EFFECTIVEBANDWIDTH')
# bandR_E = IMD.find('BAND_R').findtext('EFFECTIVEBANDWIDTH')
# bandRE_E = IMD.find('BAND_RE').findtext('EFFECTIVEBANDWIDTH')
# bandN_E = IMD.find('BAND_N').findtext('EFFECTIVEBANDWIDTH')
# bandN2_E = IMD.find('BAND_N2').findtext('EFFECTIVEBANDWIDTH')
#
# gain = 0.964
# offset = -3.021
#
# # Multiply by corresponding coefficients
# band_red_c = gain * band_red * (float(bandR_E) / float(bandR_E)) + offset
# band_nir_c = gain * band_nir * (float(bandN_E)/float(bandN_E)) +offset
# band_nir2_c = gain * band_nir2 * (float(bandN2_E)/float(bandN2_E)) +offset